In [2]:
# Import dependencies
import subprocess
import json
from dotenv import load_dotenv


In [3]:
from constants import *
from web3 import Web3
from eth_account import Account
from bit import wif_to_key
from bit import PrivateKeyTestnet
from bit.network import NetworkAPI
from web3.auto.gethdev import w3
from web3.middleware import geth_poa_middleware
# from  web3.middleware import medium_gas_pricestrategy

In [105]:
from web3.gas_strategies.time_based import medium_gas_price_strategy


In [4]:
import os

In [5]:
# !pip install python-dotenv

In [6]:
# Load and set environment variables
load_dotenv('block.env')
mnemonic=os.getenv("mnemonic")

In [7]:
# mnemonic
type(mnemonic)

str

In [8]:
# Import constants.py and necessary functions from bit and web3
# YOUR CODE HERE
from constants import *

In [9]:
# command = 'php C:/Users/Hassan/Dropbox/GithubRepo/ColumbiaFintechLab/HomeWork/hw_19_python_blockchain/wallet/hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="barrel attack mammal crash expect note alcohol offer then worth kid current" --cols=path,address,privkey,pubkey --format=json'

In [19]:
# Create a function called `derive_wallets`
def derive_wallets(Mnemonic, Coin, Numderive, Format  ):
    command = f'php ./hd-wallet-derive/hd-wallet-derive.php -g --mnemonic="{Mnemonic}" --cols=path,address,privkey,pubkey --coin="{Coin}" --numderive="{Numderive}" --format="{Format}"'
    p = subprocess.Popen(command, stdout=subprocess.PIPE, shell=True)
    output, err = p.communicate()
    p_status = p.wait()
    return json.loads(output)

In [114]:
# Create a dictionary object called coins to store the output from `derive_wallets`.
coins = {
    ETH: derive_wallets(mnemonic, ETH, 3,  'json'),
    BTCTEST: derive_wallets(mnemonic, BTCTEST, 3,  'json')
}

In [12]:
mnemonic

'barrel attack mammal crash expect note alcohol offer then worth kid current'

In [115]:
coins

{'eth': [{'path': "m/44'/60'/0'/0/0",
   'address': '0x4777eB1A1ad29C435b9051158E60314CB0C60f96',
   'privkey': '0xe59a9585e052a4a7d2e0e85d71cf15b531904f9846a002dea70b2a4ad33ba3fc',
   'pubkey': '03be5263fc23b0db31451bae011618b5472ee6577b1a63acdfad3982b25300d964'},
  {'path': "m/44'/60'/0'/0/1",
   'address': '0x70Bd998C8E5b68A86B0666Ec4D33ace4AdAb4e68',
   'privkey': '0xe7a541d579efb2beecf7d4e810aaf62d624bb7b4984cd6b9eec34e7c4964b733',
   'pubkey': '030ddf97232fa7e4e125f6e24768c39d8ce44e92887d49c5d159726ac29fa8bb28'},
  {'path': "m/44'/60'/0'/0/2",
   'address': '0x341d658cE19663a5A43987162618838dEBC443D6',
   'privkey': '0xd82c9c478fd0a26af73b108b2121252e842943e93c135dc0c813981e3e84b11c',
   'pubkey': '02cb0fef7919d0f5c4b34e78e28b7389cfbf99efedf51775570dd410679d60ff4b'}],
 'btc-test': [{'path': "m/44'/1'/0'/0/0",
   'address': 'ms4CqLw5Gz1z18ESxy4KXcgn5sWYfxnhzx',
   'privkey': 'cQ9GFdiHmRGVYi5Pym3ndA2omGQR2hWUy6rxAFiHVfWHYDtnjZb2',
   'pubkey': '02c32877af96f81a89174e36a89474a5998ce

In [14]:
# delete
# coins

In [79]:
# Create a function called `priv_key_to_account` that converts privkey strings to account objects.
def priv_key_to_account(coin, priv_key):
    # YOUR CODE HERE
    if coin == ETH:
        return Account.privateKeyToAccount(priv_key)
    if coin == BTCTEST:
        return PrivateKeyTestnet(priv_key)
        
    else:
        return ('error')

In [116]:
#delete test code
eth_coins= coins['eth']
eth1 = eth_coins[0]
eth_prv_key1 = eth1['privkey']
# eth_prf_key1

Eth_act1 = priv_key_to_account(ETH, eth_prv_key1)
Eth_act1

In [81]:
eth1['address']

'0x4777eB1A1ad29C435b9051158E60314CB0C60f96'

In [24]:
# debug test code
BTC_prv_key1 = coins ['btc-test'][0]['privkey']
BTC_prv_key1
BTC_act1 = priv_key_to_account ('btc-test',BTC_prv_key1 )
BTC_act1.address

'ms4CqLw5Gz1z18ESxy4KXcgn5sWYfxnhzx'

In [101]:
# Create a function called `create_tx` that creates an unsigned transaction appropriate metadata.
def create_tx(coin, account, recipient, amount):

    if coin == ETH:
       # need to fix parameters
        print (f'from address: {account.address}\n')
        value = w3.toWei(amount, "ether") 
        gasEstimate = w3.eth.estimateGas(
            {"from": account.address, "to": recipient, "value": value}
            )
        return {
            "from": account.address,
            "to": recipient,
            "value": value,
            "gasPrice": w3.eth.gasPrice,
            "gas": gasEstimate,
            "nonce": w3.eth.getTransactionCount(account.address),
            }
    
    if coin == BTCTEST:
        return PrivateKeyTestnet.prepare_transaction(account.address, [(recipient, amount, BTC)])
    else:
        return ('error')
        

## <span style="color:red">run till here</span>

In [ ]:
# test with btc delete
BTC_adr2 = coins ['btc-test'][1]['address']
# def create_tx(coin, account, recipient, amount):
BTC_tx = create_tx ('btc-test', BTC_act1, BTC_adr2, 0.000001)
BTC_tx

In [106]:
# delete test code
eth_adr2 = coins['eth'][1]['address']
# eth_ret = create_tx(ETH,Eth_act1, eth_adr2, 0.01  )
# w3 = Web3(Web3.HTTPProvider('http://localhost:8545'))
# HTTP://127.0.0.1:8545
w3 = Web3(Web3.HTTPProvider('http://127.0.0.1:8545'))
account = Eth_act1
recipient = eth_adr2
amount = 2.0
w3.middleware_onion.inject(geth_poa_middleware, layer=0)
w3.eth.setGasPriceStrategy(medium_gas_price_strategy)


In [24]:
# delete test code
# recipient

'0x70Bd998C8E5b68A86B0666Ec4D33ace4AdAb4e68'

In [28]:
# delete test code
# gasEstimate = w3.eth.estimateGas(
#             {"from": account.address, "to": recipient, "value": int(amount)}
#             )

In [48]:
# Create a function called `send_tx` that calls `create_tx`, signs and sends the transaction.
def send_tx(coin, account, recipient, amount):
    # YOUR CODE HERE
    tx = create_tx(coin, account, recipient, amount)
    signed_tx = account.sign_transaction(tx)
    if coin == ETH:
        result = w3.eth.sendRawTransaction(signed_tx.rawTransaction)
        return result

    if coin == BTCTEST:      
        return NetworkAPI.broadcast_tx_testnet(signed_tx)
    else:
        return ('error')
        

In [29]:
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [35]:
print (f'Ethat act 1 : {Eth_act1}\n')
print (f'address 2: {eth_adr2}')

Ethat act 1 : <eth_account.signers.local.LocalAccount object at 0x000001FFB01537C8>

address 2: 0x70Bd998C8E5b68A86B0666Ec4D33ace4AdAb4e68


In [107]:
tx = create_tx(ETH, Eth_act1, eth_adr2,.000000001)
print (f'transation: {tx}')


from address: 0x4777eB1A1ad29C435b9051158E60314CB0C60f96

transation: {'from': '0x4777eB1A1ad29C435b9051158E60314CB0C60f96', 'to': '0x70Bd998C8E5b68A86B0666Ec4D33ace4AdAb4e68', 'value': 1000000000, 'gasPrice': 1000000000, 'gas': 21000, 'nonce': 0}


In [118]:
w3.eth.getBalance("0x4777eB1A1ad29C435b9051158E60314CB0C60f96")


0

In [92]:
eth_coins= coins['eth']
eth1 = eth_coins[0]
eth_prv_key1 = eth1['privkey']
# eth_prf_key1

Eth_act1 = priv_key_to_account(ETH, eth1['privkey'])
print (f'Eth 1 Account : {Eth_act1}\n')
print (f'Eth 1 private key: {eth1["privkey"]}\n')
print (f'Eth 1 address: {eth1["address"]}\n')
print (f'Eth 2 address: {eth_adr2}')

Eth 1 Account : <eth_account.signers.local.LocalAccount object at 0x000001FFB0206C08>

Eth 1 private key: 0xe59a9585e052a4a7d2e0e85d71cf15b531904f9846a002dea70b2a4ad33ba3fc

Eth 1 address: 0x4777eB1A1ad29C435b9051158E60314CB0C60f96

Eth 2 address: 0x70Bd998C8E5b68A86B0666Ec4D33ace4AdAb4e68


In [ ]:
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [117]:
# delete test code
eth_send = send_tx(ETH,Eth_act1, eth_adr2,.0000000001 )
eth_send

from address: 0x4777eB1A1ad29C435b9051158E60314CB0C60f96



ValueError: {'code': -32000, 'message': 'insufficient funds for gas * price + value'}

In [ ]:
# delete test code
eth_send = send_tx(ETH,Eth_act1, eth_adr2,1 )
eth_send

In [56]:
btc_send = send_tx('btc-test', BTC_act1, BTC_adr2, 0.0001)
btc_send

In [55]:
BTC_act1
BTC_adr2

'mgLQNbnYcBHcgY5sCAWpQWQacznN3fDzQP'

In [57]:
loc_eth_adr = '0x20AB545aFF9eDA6841C885D0027b978cd18d9bd5'
net_name = 'puppethonastring'
w3.middleware_onion.inject(geth_poa_middleware, layer=0)

In [ ]:
# http://127.0.0.1:8545/
puppethonastring.json